In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [ ]:
# Initialising 
mp_drawings = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle


In [7]:
camera = cv2.VideoCapture(0)
counter = 0
stage = None

while camera.isOpened():
    status, frame = camera.read()
    if status == False:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(image)
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        angle = calculate_angle(shoulder, elbow, wrist)

        # Curl counter logic
        if angle > 160:
            stage = "down"
        if angle < 50 and stage == "down":
            stage = "up"
            counter += 1

        # Show counter
        cv2.putText(image, f'Reps: {counter}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        mp_drawings.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Physical Therapy Tracker', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


In [ ]:
# import cv2
# import mediapipe as mp
import csv
import os

mp_pose = mp.solutions.pose
mp_drawings = mp.solutions.drawing_utils

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

camera = cv2.VideoCapture(0)

# Create CSV file to store landmarks
csv_file = "exercise_data.csv"
file_exists = os.path.isfile(csv_file)

with open(csv_file, mode='a', newline='') as f:
    csv_writer = csv.writer(f)

    # Write header only if file is new
    if not file_exists:
        header = ["exercise", "frame"]
        for i in range(33):  # Mediapipe has 33 landmarks
            header += [f"x{i}", f"y{i}", f"z{i}", f"v{i}"]  # visibility
        csv_writer.writerow(header)

    frame_count = 0
    exercise_name = "bicep_curl"  # 🔥 Change this when recording other exercises

    while camera.isOpened():
        status, frame = camera.read()
        if not status:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Collect landmark data
            row = [exercise_name, frame_count]
            for lm in landmarks:
                row.extend([lm.x, lm.y, lm.z, lm.visibility])

            # Save to CSV
            csv_writer.writerow(row)
            frame_count += 1

            # Draw pose on screen
            mp_drawings.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Data Collection - Press Q to stop', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()
